In [3]:
DB_SETTINGS = {
    'source_db_localhost' : {
        'location' : 'localhost_source',
        'host' : "127.0.0.1",
        'port' : "5432",
        'user' : "postgres",
        'password' : "208300",
        'database' : "dvdrental",
        'engine' : 'postgre'
    }
}
DB_SETTINGS['source_db_localhost']


##########


import psycopg2 as postgre
#import pymssql as mssql
#import pymysql as mysql

import pandas as pd

class DBConnector:
    def __init__(self, location, host, port, user, password, database, engine):
        self.conn_params = dict(
            host = host,
            port = port,
            user = user,
            password = password,
            database = database
        )
        # self.conn = None

    def _postgre_connect(self):
        conn = postgre.connect(**self.conn_params)
        return conn

db = DBConnector(**DB_SETTINGS['source_db_localhost'])
db._postgre_connect()


##########


batch_month = '201305'

queries = {
    'read': {
        'actor': f'''
            SELECT '{batch_month}' AS YYYYMM, * 
            FROM actor 
            WHERE TO_CHAR(CAST(last_update AS DATE), 'YYYYMM') = '{batch_month}'
            ;
        ''',
        'film': f'''
            SELECT '{batch_month}' AS YYYYMM, * 
            FROM film 
            WHERE TO_CHAR(CAST(last_update AS DATE), 'YYYYMM') = '{batch_month}'
            ;
        '''}
    }

print(queries['read']['actor'])


##########


#cur = db._postgre_connect().cursor()
#cur.execute(queries['read']['actor'])

#result_all = cur.fetchall()
#print(result_all[:5])

#pdf = pd.DataFrame(result_all)
#pdf.head()


##########


pdf_sql = pd.read_sql(queries['read']['actor'], db._postgre_connect())
display(pdf_sql.head())
pdf_sql.info()


            SELECT '201305' AS YYYYMM, * 
            FROM actor 
            WHERE TO_CHAR(CAST(last_update AS DATE), 'YYYYMM') = '201305'
            ;
        


C:\Users\user\AppData\Local\Temp\ipykernel_3376\4114771293.py:83: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  pdf_sql = pd.read_sql(queries['read']['actor'], db._postgre_connect())


,yyyymm,actor_id,first_name,last_name,last_update
0,201305,1,Penelope,Guiness,2013-05-26 14:47:57.620
1,201305,2,Nick,Wahlberg,2013-05-26 14:47:57.620
2,201305,3,Ed,Chase,2013-05-26 14:47:57.620
3,201305,4,Jennifer,Davis,2013-05-26 14:47:57.620
4,201305,5,Johnny,Lollobrigida,2013-05-26 14:47:57.620


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 200 entries, 0 to 199
Data columns (total 5 columns):
 #   Column       Non-Null Count  Dtype         
---  ------       --------------  -----         
 0   yyyymm       200 non-null    object        
 1   actor_id     200 non-null    int64         
 2   first_name   200 non-null    object        
 3   last_name    200 non-null    object        
 4   last_update  200 non-null    datetime64[ns]
dtypes: datetime64[ns](1), int64(1), object(3)
memory usage: 7.9+ KB


In [7]:
_dtype = pd.read_sql_query("""SELECT column_name, data_type FROM information_schema."columns" WHERE table_name = 'film';""", db._postgre_connect())
_dtype

C:\Users\user\AppData\Local\Temp\ipykernel_3376\12080948.py:1: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  _dtype = pd.read_sql_query("""SELECT column_name, data_type FROM information_schema."columns" WHERE table_name = 'film';""", db._postgre_connect())


,column_name,data_type
0,fulltext,tsvector
1,rating,USER-DEFINED
2,last_update,timestamp without time zone
3,film_id,integer
4,release_year,integer
5,language_id,smallint
6,rental_duration,smallint
7,rental_rate,numeric
8,length,smallint
9,replacement_cost,numeric


In [8]:
_dtype.values

array([['fulltext', 'tsvector'],
       ['rating', 'USER-DEFINED'],
       ['last_update', 'timestamp without time zone'],
       ['film_id', 'integer'],
       ['release_year', 'integer'],
       ['language_id', 'smallint'],
       ['rental_duration', 'smallint'],
       ['rental_rate', 'numeric'],
       ['length', 'smallint'],
       ['replacement_cost', 'numeric'],
       ['title', 'character varying'],
       ['description', 'text'],
       ['special_features', 'ARRAY']], dtype=object)

In [15]:
_bool = _dtype[_dtype.data_type.str.contains('bit', regex = True)].copy()
_bool
_bool['data_type']

Series([], Name: data_type, dtype: object)

In [14]:
_int = _dtype[_dtype.data_type.str.contains('tinyint|smallint|int|bigint|NUMBER', regex = True)].copy()
_int

,column_name,data_type


In [10]:

_bool = _dtype[_dtype.data_type.str.contains('bit', regex = True)].copy()
_bool['data_type'] = 'float64' # bit type을 True/False 값이 아닌 float 으로 초기 선언 (0.0, 1.0, null)
# bool 선언시 null == False, int64 선언시 null 허용하지 않아 오류 남, Int64 선언시 bit type의 값이 True/False의 string으로 read 되어 integertype 으로 converting 불가

_int = _dtype[_dtype.data_type.str.contains('tinyint|smallint|int|bigint|NUMBER', regex = True)].copy()
_int['data_type'] = 'float64' # int type을 null 허용하는 float 으로 초기 선언 (float, null)
# int64 선언시 null 허용하지 않아 오류 남, Int64 선언해도 되지만 안전성을 위해 float으로 초기값 선언

_float = _dtype[_dtype.data_type.str.contains('float|real', regex = True)].copy()
_float['data_type'] = 'float64'

_datetime = _dtype[_dtype.data_type.str.contains('date|datetime|datetime2|DATE', regex = True)].copy()
_datetime['data_type'] = 'datetime64'

_dtype = pd.concat([_bool, _int, _float, _datetime])

pandas_df = pd.read_sql_query(_query, connected.conn, dtype = dict(_dtype.values))

pandas_df[_bool.COLUMN_NAME] = pandas_df[_bool.COLUMN_NAME].astype(pd.Int64Dtype()) # float 으로 선언된 bool type을 null 허용하는 pd.Int64Dtype()으로 변환, np.int64 사용하면 null 허용하지 않아 오류 남
pandas_df[_int.COLUMN_NAME] = pandas_df[_int.COLUMN_NAME].astype(pd.Int64Dtype()) # float 으로 선언된 int type을 null 허용하는 pd.Int64Dtype()으로 변환, np.int64 사용하면 null 허용하지 않아 오류 남
            
for col in pandas_df.select_dtypes('object').columns:
                pandas_df[col] = pandas_df[col].astype(str)

NameError: name '_query' is not defined